In [ ]:
!pip install pyshp
!pip install geopandas
!pip install --upgrade nbformat
## Ensure running conda install --channel conda-forge cartopy before
!pip install geoplot


!pip install PyBNG

In [ ]:
import geopandas as gpd
import glob
import pandas as pd

# Read SHx files



## Grid references for London: https://getoutside.ordnancesurvey.co.uk/guides/beginners-guide-to-grid-references/
files = [
    "SP_RoadNode.shx",
    "TL_RoadNode.shx",
    "SU_RoadNode.shx",
    "TQ_RoadNode.shx"
]

path = "data_storage/data_storage/oproad_essh_gb/data/"

paths = [path+file for file in files]

geo_df = pd.concat(map(gpd.read_file, paths))



#lga_gdf = gpd.read_file('data_storage/data_storage/oproad_essh_gb/data/SP_RoadNode.shx')    # Load the data using Geopandas

In [ ]:
type(geo_df)

In [ ]:
geo_df.geometry.x/1e4

In [ ]:
geo_df.head(1000)

In [ ]:
geo_df.dtypes

In [ ]:
vals = geo_df.geometry.x/1e4
vals.describe()

In [ ]:
vals = geo_df.geometry.y/1e4
vals.describe()

In [ ]:
geo_df["formOfNode"].unique()

In [ ]:
geo_df["formOfNode"].value_counts()

In [ ]:
geo_df.count()

### Conversion 

In [ ]:
def convert_bng_to_wsg(coord):
    
    print(coord.geometry.x*100, coord.geometry.y*100)

    bng = PyBNG(easting=int(coord.geometry.x*1000), northing=int(coord.geometry.y*1000))
    coord["latitude"], coord["longitude"] = bng.get_latlon()
#     coord["latitude"]*=0.9996012717
#     coord["longitude"]*=0.9996012717

    return coord

In [ ]:
from PyBNG import PyBNG




import plotly.express as px
import geopandas as gpd
import plotly.io as pio
pio.renderers.default = 'browser'

sample = geo_df.sample(n=1000)
sample = sample.apply(convert_bng_to_wsg, axis=1)

In [ ]:
fig = px.scatter_mapbox(sample,
    lat=sample.latitude,
    lon=sample.longitude,
    hover_name="formOfNode",
    zoom=14
)

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.write_html("output/road_features.html")
fig.show()